In [ ]:
# Optional: install packages if not using Docker images
# !pip install datasets pandas numpy matplotlib seaborn scikit-learn sentence-transformers chromadb joblib tqdm

In [ ]:
# Imports
import sys, os
import pandas as pd
import numpy as np
import matplotlib
import seaborn
import sklearn
from datasets import load_dataset
from sentence_transformers import SentenceTransformer
import chromadb
import joblib
print("✅ Imports OK")

In [ ]:
# Print versions for interview reproducibility
print("Python:", sys.version)
print("pandas:", pd.__version__)
print("numpy:", np.__version__)
print("matplotlib:", matplotlib.__version__)
print("seaborn:", seaborn.__version__)
print("scikit-learn:", sklearn.__version__)
print("chromadb:", chromadb.__version__)
print("sentence-transformers loaded")

In [ ]:
# Prepare workspace folders
for p in ['../data', '../models', '../chromadb_data']:
    os.makedirs(p, exist_ok=True)
print("📁 Folders ready: data/models/chromadb_data")

## Next Steps
- Open `01_batch_processing.ipynb` to perform EDA and training.
- Keep this notebook focused on environment checks (no EDA here).